In [1]:
import json
import pandas as pd


"""
TODO:
1. Data sudah bener2 post_canonical, tinggal processing untuk di training BERT
2. Tambah new column untuk ini mungkin namanya `post_processed`
"""

def remove_redundant_white_spaces(text: str) -> str:
    from re import sub
    return sub(r'\s+', ' ', text).strip()


file_path = 'C:/wamp64/www/artofproblemsolving/data/data_post_canonical.csv'
df = pd.read_csv(file_path)


In [55]:
def processing_dollar_sign(text: str) -> str:
    def findall_occurrences_of_LaTex(text: str) -> list:
        """
        Find/capture all occurrences LaTex that indicated with `$ ... $` with an n number of `$`
        """
        import re

        regex = re.compile(r'(\$+.*?\$+)')
        matches = re.findall(regex, text)
        
        return matches

    def extract_inside_dollar_sign(text: str) -> str:
        """
        This function only for this specific input:
        - `$ 1/x \leq 10 $`  -->  ` 1/x \leq 10 `
        - `$1  anything z$`  -->  `1   anything z`  with the redundant spaces

        You can say this function remove suffix and prefix for `$` sign\n
        OR\n
        It takes/extracts text that flanked (diapit) with `$` sign
        """
        import re
        
        regex = re.compile(r'\$+(.*?)\$+')
        result = re.findall(regex, text)
        return result[0]


    def replace_le_leq_leqslant_with_LESSEQUALTHANSIGN(input_string) -> str:
        import re

        # \le \leq \leqslant
        le_patterns = [r'\\leqslant', r'\\leq', r'\\le']

        for pattern in le_patterns:
            input_string = re.sub(pattern, ' LESSEQUALTHANSIGN ', input_string)

        return remove_redundant_white_spaces(input_string)

    def replace_ge_geq_geqslant_with_GREATEREQUALTHANSIGN(input_string) -> str:
        import re
        
        # \ge \geq \geqslant
        ge_patterns = [r'\\geqslant', r'\\geq', r'\\ge']

        for pattern in ge_patterns:
            input_string = re.sub(pattern, ' GREATEREQUALTHANSIGN ', input_string)

        return remove_redundant_white_spaces(input_string)

    def replace_cdots_ldots_hdots_dots_with_THREEDOTS(input_string) -> str:
        import re
        
        # \cdots \ldots \hdots \dots
        dots_patterns = [r'\\cdots', r'\\ldots', r'\\hdots', r'\\dots']
        
        for pattern in dots_patterns:
            input_string = re.sub(pattern, ' THREEDOTS ', input_string)
        
        return remove_redundant_white_spaces(input_string)

    def replace_neq_ne_with_NOTEQUALTOSIGN(input_string) -> str:
        import re
        
        # \ne \neq
        ne_patterns = [r'\\neq', r'\\ne']
        
        for pattern in ne_patterns:
            input_string = re.sub(pattern, ' NOTEQUALTOSIGN ', input_string)
        
        return remove_redundant_white_spaces(input_string)
        

    def replace_to_LOWERCASESUBSCRIPTVAR(text):
        """
        - a_i
        - a_2
        - a_{i+2}
        - a_{i+j}
        """
        import re
        
        regex = re.compile(r'[a-z]_({.*?}|[a-zA-Z\d^}])')
        result = regex.sub(' LOWERCASESUBSCRIPTVAR ', text)
        
        return result
    
    def replace_to_UPPERCASESUBSCRIPTVAR(text):
        """
        - A_i
        - A_2
        - A_{i+2}
        - A_{i+j}
        """
        import re
        
        regex = re.compile(r'[A-Z]_({.*?}|[a-zA-Z\d^}])')
        result = regex.sub(' UPPERCASESUBSCRIPTVAR ', text)
        
        return result
    
    
    def replace_to_LOWERCASESUPERSCRIPTVAR(text):
        import re

        regex = re.compile(r'[a-z]\^')

    def replace_to_UPPERCASESUPERSCRIPTVAR(text):...
    
    
    def replace_to_LOWERCASEVAR(text):...

    def replace_to_UPPERCASEVAR(tex):...
        
    # \quad \qquad 
    # \text \textrm \text.. ...
    # \mathrm \mathbf \mathsf ...
    
    # 1. First: Case for some LaTex that look like this `\[ ... \]` need to replaced with `$`
    text = text.replace("\[", "$")
    text = text.replace("\]", "$")
    
    # 2. Capture all Latex indicated with flanked `$`. Example: `$ ... $`
    captured_LaTex = findall_occurrences_of_LaTex(text)
    processed_LaTex = captured_LaTex.copy()
    
    # 3. Pre-process the captured LaTex
    for i in range(len(processed_LaTex)):
        processed_LaTex[i] = extract_inside_dollar_sign(processed_LaTex[i])
        
        processed_LaTex[i] = replace_le_leq_leqslant_with_LESSEQUALTHANSIGN(processed_LaTex[i])
        processed_LaTex[i] = replace_ge_geq_geqslant_with_GREATEREQUALTHANSIGN(processed_LaTex[i])
        processed_LaTex[i] = replace_cdots_ldots_hdots_dots_with_THREEDOTS(processed_LaTex[i])
        processed_LaTex[i] = replace_neq_ne_with_NOTEQUALTOSIGN(processed_LaTex[i])
        processed_LaTex[i] = replace_to_LOWERCASESUBSCRIPTVAR(processed_LaTex[i])
        
        processed_LaTex[i] = processed_LaTex[i].replace(">", " GREATERTHANSIGN ")
        processed_LaTex[i] = processed_LaTex[i].replace("<", " LESSTHANSIGN ")
        processed_LaTex[i] = processed_LaTex[i].replace("+", " PLUSORPOSITIVESIGN ")
        processed_LaTex[i] = processed_LaTex[i].replace("-", " MINUSORNEGATIVESIGN ")
        processed_LaTex[i] = processed_LaTex[i].replace("=", " EQUALTOSIGN ")
        processed_LaTex[i] = processed_LaTex[i].replace("!", " FACTORIALSIGN ")
        
    # 4. Finally, put them back together (replace them)
    for i in range(len(captured_LaTex)):  # Same thing using `len(processed_LaTex)`
        text = text.replace(captured_LaTex[i], processed_LaTex[i])
    
    # Last processing
    # NOTE: removing `\` is the last part because... (it's my intuision hehe)
    text = text.replace("\\", " ")
    text = remove_redundant_white_spaces(text)
    
    return text
    

In [56]:
# contain unicode
t = (df[df["year"] == "2014_imo"][df[df["year"] == "2014_imo"]["no"] == "1"]).iloc[0]["post_canonical"]

print(t)
print()
print(processing_dollar_sign(t))


Let $a_0 < a_1 < a_2 \ldots$ be an infinite sequence of positive integers. Prove that there exists a unique integer $n\geq 1$ such that \[a_n < \frac{a_0+a_1+a_2+\cdots+a_n}{n} \leq a_{n+1}.\]  Proposed by Gerhard Wöginger, Austria.

Let LOWERCASESUBSCRIPTVAR LESSTHANSIGN LOWERCASESUBSCRIPTVAR LESSTHANSIGN LOWERCASESUBSCRIPTVAR THREEDOTS be an infinite sequence of positive integers. Prove that there exists a unique integer n GREATEREQUALTHANSIGN 1 such that LOWERCASESUBSCRIPTVAR LESSTHANSIGN frac{ LOWERCASESUBSCRIPTVAR PLUSORPOSITIVESIGN LOWERCASESUBSCRIPTVAR PLUSORPOSITIVESIGN LOWERCASESUBSCRIPTVAR PLUSORPOSITIVESIGN THREEDOTS PLUSORPOSITIVESIGN LOWERCASESUBSCRIPTVAR }{n} LESSEQUALTHANSIGN LOWERCASESUBSCRIPTVAR . Proposed by Gerhard Wöginger, Austria.


In [13]:
from random import randint

n = randint(0, 10000)
# n = 9915
# n = 4501
test_ = df.iloc[n]['post_canonical']
link_ = df.iloc[n]['link']

print(n)
print(link_)
print(test_)
print(processing_dollar_sign(test_))


9238
https://artofproblemsolving.com/community/c618724_2018_romanian_masters_in_mathematics
Let $n$ be positive integer and fix $2n$ distinct points on a circle. Determine the number of ways to connect the points with $n$ arrows (oriented line segments) such that all of the following conditions hold:  each of the $2n$ points is a startpoint or endpoint of an arrow; no two arrows intersect; and there are no two arrows $\overrightarrow{AB}$ and $\overrightarrow{CD}$ such that $A$, $B$, $C$ and $D$ appear in clockwise order around the circle (not necessarily consecutively). 
Let n be positive integer and fix 2n distinct points on a circle. Determine the number of ways to connect the points with n arrows (oriented line segments) such that all of the following conditions hold: each of the 2n points is a startpoint or endpoint of an arrow; no two arrows intersect; and there are no two arrows overrightarrow{AB} and overrightarrow{CD} such that A, B, C and D appear in clockwise order around th

In [ ]:
# Make a new column first name `post_training`

for i in range(df.shape[0]):
    df.iloc[i]['']
    ...
    

In [7]:
# / == \frac{}{}
# :

# ^
superscript_variable = "superscript_variable"
# _
subscript_variable = "subscript_variable"
# ^_
combined_supersubscript_variable = "combined_supersubscript_variable"

# ABC --> "uppercase_variable uppercase_variable uppercase_variable"
uppercase_variable = "uppercase_variable"
# n, x, y --> lowercase_variable
lowercase_variable = "lowercase_variable"
